In [4]:
import datetime as dt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

In [6]:
data = pd.read_csv('data/preprocessed_data.csv')

data.head()

,Unnamed: 0,occur_month,occur_day,occur_time,location,victims,day_of_week,crime,neighborhood,hour_0.0,...,month_11,month_12,week_0,week_1,week_2,week_3,week_4,week_5,week_6,week_7
0,0,12,30,23.0,2014,2.0,2,5,78,0,...,0,1,0,0,1,0,0,0,0,0
1,1,12,18,13.0,355,1.0,6,5,115,0,...,0,1,0,0,0,0,0,0,1,0
2,2,12,30,22.0,3473,1.0,2,5,143,0,...,0,1,0,0,1,0,0,0,0,0
3,3,12,30,20.0,2858,1.0,2,5,62,0,...,0,1,0,0,1,0,0,0,0,0
4,4,12,31,0.0,813,2.0,3,5,1,1,...,0,1,0,0,0,1,0,0,0,0


In [7]:
learning_rate = 0.1
num_steps = 500
batch_sizee = 128
display_step = 100

In [11]:
n_hidden_1 = 256
n_hidden_2 = 256
num_input = 83
num_classes = 11

# data.crime.nunique()
X = tf.placeholder(tf.int8, [None, 83])
y = tf.placeholder(tf.int8, [None, 11])

In [7]:
X = data.drop(['crime'], axis=1)
y = data['crime']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 4)